In [7]:
import time
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from functools import reduce
import time
import selenium
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.options import Options

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

### Selenium navigation

In [ ]:
a = pd.DataFrame()

In [96]:
a = ['pbp', 'advanced', 'shooting']
s = '_'.join(a)
s

'pbp_advanced_shooting'

In [137]:
def select_season(browser, season):
    url = f'https://www.basketball-reference.com/leagues/NBA_{season}_per_poss.html'
    browser.get(url)

def select_stat_table(browser, stat_table):
    ### DEPENDS ON YEAR ... E.G. OLDER STATS HAVE NO PLAY-BY-PLAY SO NUMBERS ARE DIFF
    ## key names are such that they match bball ref internal reference for links and stuff
    stat_table_dict = {'totals': 1, 'per_game': 2, 'per_minute': 3, 'per_poss': 4, 'advanced': 5, 'pbp': 6, 'shooting': 7, 'adj_shooting': 8}
    n = stat_table_dict[stat_table]
    button_xpath = f'/html/body/div[2]/div[5]/div[2]/div[{n}]/a'
    browser.find_element_by_xpath(button_xpath).click()
    
def toggle_partial_rows(browser, stat_type):
    button_xpath = f'//*[@id="{stat_type}_stats_toggle_partial_table"]'
    browser.find_element_by_xpath(button_xpath).click()

def remove_mobile_fmt(browser):
    action = ActionChains(browser)
    menu_xpath =  '/html/body/div[2]/div[5]/div[3]/div[1]/div/ul/li[1]/span'
    share_menu = browser.find_element_by_xpath(menu_xpath)
    action.move_to_element(share_menu).perform(); time.sleep(5)
    button_xpath = '/html/body/div[2]/div[5]/div[3]/div[1]/div/ul/li[1]/div/ul/li[5]/button'
    browser.find_element_by_xpath(button_xpath).click()
    
def read_stat_table(soup):
    table = soup.find(class_="table_outer_container")
    tbody = table.find('tbody')
    tbody_tr = tbody.find_all('tr')
    stat_cols, all_stats, all_ids = [], [], []
    
    for row in tbody_tr:
        if row['class'] != ["full_table"]: continue  # skip partial entries for players with multiple teams
        cols = row.find_all('td')
        if stat_cols == []:  # 
            stat_cols = [col.get('data-stat') for col in cols]
        player_stats = [col.text for col in cols]
        all_stats.append(player_stats)
        player_id = cols[0].get('data-append-csv')
        all_ids.append(player_id)

    df = pd.DataFrame(all_stats, columns=stat_cols).set_index('player')
    df['player_id'] = all_ids
    
    return df

def process_scraped_table(df, season):
    drop_cols = [col for col in df.columns if df[col].nunique() == 1]
    df.drop(drop_cols, axis=1, inplace=True)
    df['year'] = [season] * len(df)
    df.index += f' {str(season)}'
    return df

def combine_season_stats(dfs):
    
#     print(dfs)
    
    dfs = [df.reset_index().set_index('player_id') for df in dfs]
    season_df = reduce(lambda left, right: pd.merge(left, right[right.columns.difference(left.columns)],
                                                   left_index=True, right_index=True), dfs)
    return season_df.reset_index().set_index('player')

def scrape(seasons, stat_types, data_dir):
    options = webdriver.firefox.options.Options()
    options.headless = True
    browser = webdriver.Firefox(executable_path="./drivers/geckodriver", options=options)
    dfs = []
    
    for season in seasons:
        print('scraping:', season)
        season_dfs = []
        select_season(browser, season); time.sleep(3)
        
        for stat_type in stat_types:
            select_stat_table(browser, stat_type); time.sleep(3)
            page_source = browser.page_source
            soup = BeautifulSoup(page_source, 'html.parser')
            df = read_stat_table(soup)
            df = process_scraped_table(df, season)        
            filename = data_dir + str(season) + '_' + str(stat_type).replace(' ', '_') + '.csv'
            df.to_csv(filename)
            season_dfs.append(season_df)
    
        season_df = combine_season_stats(season_dfs)
        filename = data_dir + str(season) + '_'.join(stat_types) + '.csv'
        season_df.to_csv(filename)
        dfs.append(season_df)
    
    if len(seasons) < 1:
        return dfs[0]
    
    master_df = pd.concat(dfs)
    filename = data_dir + str(season[0]) + '_' + str(season[-1]) + '_'.join(stat_types) + '.csv'
    master_df.to_csv()
    
    return master_df



In [139]:
a = set()
for df in dfs:
    a = a.union(set(df.columns))
len(a)

88

In [149]:
df = pd.read_csv('../TEST/2019_2020_advanced_shooting_pbp_per_poss.csv', index_col=0)
df.shape
df.head()

(1059, 88)

,player_id,pos,age,team_id,g,mp,per,ts_pct,fg3a_per_fga_pct,fta_per_fga_pct,...,ft_per_poss,fta_per_poss,gs,off_rtg,orb_per_poss,pf_per_poss,pts_per_poss,stl_per_poss,tov_per_poss,trb_per_poss
player,,,,,,,,,,,,,,,,,,,,,
Álex Abrines 2019,abrinal01,SG,25,OKC,31,588,6.3,0.507,0.809,0.083,...,1.0,1.0,2,103.0,0.4,4.2,13.1,1.3,1.1,3.8
Quincy Acy 2019,acyqu01,PF,28,PHO,10,123,2.9,0.379,0.833,0.556,...,2.7,3.9,0,87.0,1.2,9.3,6.6,0.4,1.6,9.7
Jaylen Adams 2019,adamsja01,PG,22,ATL,34,428,7.6,0.474,0.673,0.082,...,0.8,1.0,1,99.0,1.2,4.9,11.7,1.5,3.0,6.5
Steven Adams 2019,adamsst01,C,25,OKC,80,2669,18.5,0.591,0.002,0.361,...,2.6,5.1,80,120.0,6.8,3.6,19.4,2.0,2.4,13.3
Bam Adebayo 2019,adebaba01,C,21,MIA,82,1913,17.9,0.623,0.031,0.465,...,4.2,5.8,28,120.0,4.2,5.2,18.6,1.8,3.1,15.2


In [157]:
df['player_id'].sort_values()

player
Álex Abrines 2019    abrinal01
Quincy Acy 2019        acyqu01
Jaylen Adams 2019    adamsja01
Steven Adams 2019    adamsst01
Steven Adams 2020    adamsst01
                       ...    
Tyler Zeller 2019    zellety01
Ante Žižić 2020      zizican01
Ante Žižić 2019      zizican01
Ivica Zubac 2019     zubaciv01
Ivica Zubac 2020     zubaciv01
Name: player_id, Length: 1059, dtype: object

In [155]:
a = list(df.columns)
sorted(a)

['age',
 'and1s',
 'ast_pct',
 'ast_per_poss',
 'astd_pts',
 'avg_dist',
 'blk_pct',
 'blk_per_poss',
 'bpm',
 'dbpm',
 'def_rtg',
 'drawn_offensive',
 'drawn_shooting',
 'drb_pct',
 'drb_per_poss',
 'dws',
 'fg2_pct',
 'fg2_per_poss',
 'fg2a_per_poss',
 'fg3_heave',
 'fg3_pct',
 'fg3_per_poss',
 'fg3a_heave',
 'fg3a_per_fga_pct',
 'fg3a_per_poss',
 'fg_dunk',
 'fg_pct',
 'fg_pct_00_03',
 'fg_pct_03_10',
 'fg_pct_10_16',
 'fg_pct_16_xx',
 'fg_pct_corner3',
 'fg_pct_fg2a',
 'fg_pct_fg3a',
 'fg_per_poss',
 'fga_per_poss',
 'fouls_offensive',
 'fouls_shooting',
 'ft_pct',
 'ft_per_poss',
 'fta_per_fga_pct',
 'fta_per_poss',
 'g',
 'gs',
 'mp',
 'obpm',
 'off_rtg',
 'orb_pct',
 'orb_per_poss',
 'own_shots_blk',
 'ows',
 'pct_1',
 'pct_2',
 'pct_3',
 'pct_4',
 'pct_5',
 'pct_ast_fg2',
 'pct_ast_fg3',
 'pct_fg3a_corner3',
 'pct_fga_00_03',
 'pct_fga_03_10',
 'pct_fga_10_16',
 'pct_fga_16_xx',
 'pct_fga_dunk',
 'pct_fga_fg2a',
 'pct_fga_fg3a',
 'per',
 'pf_per_poss',
 'player_id',
 'plus_minu

In [140]:
b = combine_season_stats([df1])
b

,player_id,pos,age,team_id,g,mp,fg_pct,avg_dist,pct_fga_fg2a,pct_fga_00_03,...,fg_pct_fg3a,pct_ast_fg2,pct_ast_fg3,pct_fga_dunk,fg_dunk,pct_fg3a_corner3,fg_pct_corner3,fg3a_heave,fg3_heave,year
player,,,,,,,,,,,,,,,,,,,,,
Álex Abrines 2019,abrinal01,SG,25,OKC,31,588,0.357,22.3,0.191,0.102,...,0.323,0.800,0.951,0.006,1,0.205,0.385,2,0,2019
Quincy Acy 2019,acyqu01,PF,28,PHO,10,123,0.222,21.7,0.167,0.111,...,0.133,1.000,1.000,0.000,0,0.333,0.200,0,0,2019
Jaylen Adams 2019,adamsja01,PG,22,ATL,34,428,0.345,20.2,0.327,0.118,...,0.338,0.231,0.840,0.000,0,0.149,0.182,1,0,2019
Steven Adams 2019,adamsst01,C,25,OKC,80,2669,0.595,3.5,0.998,0.645,...,0.000,0.642,NaN,0.190,131,0.000,NaN,2,0,2019
Bam Adebayo 2019,adebaba01,C,21,MIA,82,1913,0.576,5.2,0.969,0.572,...,0.200,0.700,0.667,0.257,117,0.267,0.000,2,0,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Trae Young 2019,youngtr01,PG,20,ATL,81,2503,0.418,14.9,0.616,0.265,...,0.324,0.211,0.423,0.000,0,0.037,0.389,1,0,2019
Cody Zeller 2019,zelleco01,C,26,CHO,49,1243,0.551,5.1,0.936,0.545,...,0.273,0.745,1.000,0.139,43,0.000,NaN,0,0,2019
Tyler Zeller 2019,zellety01,C,29,TOT,6,93,0.533,4.8,0.967,0.433,...,0.000,0.625,NaN,0.067,2,1.000,0.000,0,0,2019


In [138]:
df1 = pd.read_csv('../TEST/2019_shooting.csv',index_col=0)
df2 = pd.read_csv('../TEST/2019_advanced.csv',index_col=0)
df3 = pd.read_csv('../TEST/2019_pbp.csv',index_col=0)
df4 = pd.read_csv('../TEST/2019_per_poss.csv',index_col=0)

dfs = [df1,df2,df3,df4]

df_all = combine_season_stats(dfs)
df_all.head()

,player_id,pos,age,team_id,g,mp,fg_pct,avg_dist,pct_fga_fg2a,pct_fga_00_03,...,ft_per_poss,fta_per_poss,gs,off_rtg,orb_per_poss,pf_per_poss,pts_per_poss,stl_per_poss,tov_per_poss,trb_per_poss
player,,,,,,,,,,,,,,,,,,,,,
Álex Abrines 2019,abrinal01,SG,25,OKC,31,588,0.357,22.3,0.191,0.102,...,1.0,1.0,2,103.0,0.4,4.2,13.1,1.3,1.1,3.8
Quincy Acy 2019,acyqu01,PF,28,PHO,10,123,0.222,21.7,0.167,0.111,...,2.7,3.9,0,87.0,1.2,9.3,6.6,0.4,1.6,9.7
Jaylen Adams 2019,adamsja01,PG,22,ATL,34,428,0.345,20.2,0.327,0.118,...,0.8,1.0,1,99.0,1.2,4.9,11.7,1.5,3.0,6.5
Steven Adams 2019,adamsst01,C,25,OKC,80,2669,0.595,3.5,0.998,0.645,...,2.6,5.1,80,120.0,6.8,3.6,19.4,2.0,2.4,13.3
Bam Adebayo 2019,adebaba01,C,21,MIA,82,1913,0.576,5.2,0.969,0.572,...,4.2,5.8,28,120.0,4.2,5.2,18.6,1.8,3.1,15.2


In [123]:
c = pd.DataFrame({'a': ['','','', ''], 'b': [1,2,3,0]})
c
# [col for col in c if (c[col]=='').all()]
c.any()

[col for col in c if not c[col].any()]

c['a'].any()

df = c.copy()
nunique = df.apply(pd.Series.nunique)
drop_cols = nunique[nunique == 1].index
drop_cols
c.drop(drop_cols, axis=1)

,a,b
0,,1
1,,2
2,,3
3,,0


a    False
b     True
dtype: bool

['a']

''

Index(['a'], dtype='object')

,b
0,1
1,2
2,3
3,0


In [104]:
for col in df.columns:
    print(col)
    print(df[col].nunique())
    print()

pos
13

age
22

team_id
31

g
80

mp
431

per
184

ts_pct
216

fg3a_per_fga_pct
255

fta_per_fga_pct
287

orb_pct
141

drb_pct
191

trb_pct
161

ast_pct
218

stl_pct
34

blk_pct
69

tov_pct
159

usg_pct
185

ows
85

dws
51

ws
109

ws_per_48
215

obpm
129

dbpm
70

bpm
141

vorp
63

player_id
451

year
1



In [100]:
data_dir = 'TEST/'
seasons = [2019, 2020]

stat_types = ['advanced', 'shooting', 'pbp', 'per_poss']
df = scrape(seasons, stat_types, data_dir)

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [88]:
df_combo = combine_season_stats([df1, df2], 'asd')
df_combo.shape
df_combo.head()

(451, 55)

,player_id,pos,age,team_id,g,mp,per,ts_pct,fg3a_per_fga_pct,fta_per_fga_pct,...,pct_ast_fg2,pct_ast_fg3,pct_fg3a_corner3,pct_fga_00_03,pct_fga_03_10,pct_fga_10_16,pct_fga_16_xx,pct_fga_dunk,pct_fga_fg2a,pct_fga_fg3a
player,,,,,,,,,,,,,,,,,,,,,
Shareef Abdur-Rahim 2008,abdursh01,PF,31,SAC,6,51,5.5,0.317,0.000,0.286,...,0.333,NaN,NaN,0.429,0.214,0.000,0.357,0.000,1.000,0.000
Arron Afflalo 2008,afflaar01,SG,22,DET,75,970,10.2,0.494,0.199,0.361,...,0.652,0.8,0.458,0.332,0.054,0.129,0.286,0.008,0.801,0.199
Maurice Ager 2008,agerma01,SG,23,TOT,26,165,1.8,0.363,0.292,0.185,...,0.722,1.0,0.211,0.338,0.062,0.046,0.262,0.077,0.708,0.292
Blake Ahearn 2008,ahearbl01,PG,23,MIA,12,177,9.0,0.495,0.596,0.544,...,0.200,1.0,0.235,0.140,0.070,0.088,0.105,0.000,0.404,0.596
LaMarcus Aldridge 2008,aldrila01,PF,22,POR,76,2649,18.5,0.523,0.006,0.257,...,0.645,1.0,0.143,0.197,0.272,0.218,0.308,0.079,0.994,0.006


In [98]:
df1 = pd.read_csv('TEST/2008_advanced.csv', index_col=0)
df2 = pd.read_csv('TEST/2008_shooting.csv', index_col=0)


In [89]:
df1.shape
df2.shape

(451, 27)

(451, 35)

In [90]:
len(set(df1.columns).union(set(df2.columns)) )

55

In [85]:
df1.shape
df1 = df1.reset_index()
df1.shape
df1.head()
df1 = df1.set_index('player_id')
df1.shape
df1.head()

(451, 27)

(451, 28)

,player,pos,age,team_id,g,mp,per,ts_pct,fg3a_per_fga_pct,fta_per_fga_pct,...,ows,dws,ws,ws_per_48,obpm,dbpm,bpm,vorp,player_id,year
0,Shareef Abdur-Rahim 2008,PF,31,SAC,6,51,5.5,0.317,0.000,0.286,...,0.0,0.0,0.0,-0.005,-4.4,-2.5,-6.9,-0.1,abdursh01,2008
1,Arron Afflalo 2008,SG,22,DET,75,970,10.2,0.494,0.199,0.361,...,0.5,1.4,1.9,0.092,-2.6,1.0,-1.5,0.1,afflaar01,2008
2,Maurice Ager 2008,SG,23,TOT,26,165,1.8,0.363,0.292,0.185,...,-0.4,0.0,-0.4,-0.114,-6.8,-4.2,-11.1,-0.4,agerma01,2008
3,Blake Ahearn 2008,PG,23,MIA,12,177,9.0,0.495,0.596,0.544,...,-0.1,0.1,0.0,-0.006,-3.5,-1.3,-4.8,-0.1,ahearbl01,2008
4,LaMarcus Aldridge 2008,PF,22,POR,76,2649,18.5,0.523,0.006,0.257,...,3.5,2.8,6.2,0.113,0.6,-0.4,0.2,1.5,aldrila01,2008


(451, 27)

,player,pos,age,team_id,g,mp,per,ts_pct,fg3a_per_fga_pct,fta_per_fga_pct,...,usg_pct,ows,dws,ws,ws_per_48,obpm,dbpm,bpm,vorp,year
player_id,,,,,,,,,,,,,,,,,,,,,
abdursh01,Shareef Abdur-Rahim 2008,PF,31,SAC,6,51,5.5,0.317,0.000,0.286,...,14.7,0.0,0.0,0.0,-0.005,-4.4,-2.5,-6.9,-0.1,2008
afflaar01,Arron Afflalo 2008,SG,22,DET,75,970,10.2,0.494,0.199,0.361,...,15.2,0.5,1.4,1.9,0.092,-2.6,1.0,-1.5,0.1,2008
agerma01,Maurice Ager 2008,SG,23,TOT,26,165,1.8,0.363,0.292,0.185,...,21.3,-0.4,0.0,-0.4,-0.114,-6.8,-4.2,-11.1,-0.4,2008
ahearbl01,Blake Ahearn 2008,PG,23,MIA,12,177,9.0,0.495,0.596,0.544,...,23.2,-0.1,0.1,0.0,-0.006,-3.5,-1.3,-4.8,-0.1,2008
aldrila01,LaMarcus Aldridge 2008,PF,22,POR,76,2649,18.5,0.523,0.006,0.257,...,25.3,3.5,2.8,6.2,0.113,0.6,-0.4,0.2,1.5,2008


In [63]:
df = pd.read_csv('TEST/2009_advanced.csv', index_col=0)

In [66]:
df = pd.read_csv('TEST/2009_advanced.csv', index_col=0)
print(df.shape)
df.head()


(582, 29)


,pos,age,team_id,g,mp,per,ts_pct,fg3a_per_fga_pct,fta_per_fga_pct,orb_pct,...,dws,ws,ws_per_48,bpm-dum,obpm,dbpm,bpm,vorp,player_id,year
player,,,,,,,,,,,,,,,,,,,,,
Alex Acker 2009,SG,26,TOT,25,199,10.4,0.448,0.263,0.132,4.5,...,0.1,0.0,-0.011,NaN,-1.8,-1.7,-3.6,-0.1,ackeral01,2009
Alex Acker 2009,SG,26,DET,7,20,16.0,0.379,0.364,0.182,0.0,...,0.0,0.0,0.009,NaN,-0.9,4.7,3.8,0.0,ackeral01,2009
Alex Acker 2009,SG,26,LAC,18,179,9.8,0.460,0.246,0.123,5.0,...,0.0,0.0,-0.013,NaN,-1.9,-2.5,-4.4,-0.1,ackeral01,2009
Hassan Adams 2009,SG,24,TOR,12,52,0.4,0.352,0.000,0.462,2.3,...,0.0,-0.2,-0.144,NaN,-7.5,-1.9,-9.4,-0.1,adamsha01,2009
Arron Afflalo 2009,SG,23,DET,74,1234,8.9,0.548,0.357,0.237,2.8,...,1.0,1.8,0.069,NaN,-2.8,0.2,-2.6,-0.2,afflaar01,2009


In [64]:
df

,pos,age,team_id,g,mp,per,ts_pct,fg3a_per_fga_pct,fta_per_fga_pct,orb_pct,...,dws,ws,ws_per_48,bpm-dum,obpm,dbpm,bpm,vorp,player_id,year
player,,,,,,,,,,,,,,,,,,,,,
Alex Acker 2009,SG,26,TOT,25,199,10.4,0.448,0.263,0.132,4.5,...,0.1,0.0,-0.011,NaN,-1.8,-1.7,-3.6,-0.1,ackeral01,2009
Alex Acker 2009,SG,26,DET,7,20,16.0,0.379,0.364,0.182,0.0,...,0.0,0.0,0.009,NaN,-0.9,4.7,3.8,0.0,ackeral01,2009
Alex Acker 2009,SG,26,LAC,18,179,9.8,0.460,0.246,0.123,5.0,...,0.0,0.0,-0.013,NaN,-1.9,-2.5,-4.4,-0.1,ackeral01,2009
Hassan Adams 2009,SG,24,TOR,12,52,0.4,0.352,0.000,0.462,2.3,...,0.0,-0.2,-0.144,NaN,-7.5,-1.9,-9.4,-0.1,adamsha01,2009
Arron Afflalo 2009,SG,23,DET,74,1234,8.9,0.548,0.357,0.237,2.8,...,1.0,1.8,0.069,NaN,-2.8,0.2,-2.6,-0.2,afflaar01,2009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Julian Wright 2009,SF,21,NOH,54,772,12.6,0.479,0.090,0.128,7.2,...,1.2,0.8,0.048,NaN,-1.8,0.8,-0.9,0.2,wrighju01,2009
Lorenzen Wright 2009,C,33,CLE,17,125,4.0,0.377,0.000,0.296,6.9,...,0.2,-0.1,-0.022,NaN,-7.3,0.7,-6.7,-0.1,wrighlo02,2009
Nick Young 2009,SG,23,WAS,82,1837,13.1,0.530,0.217,0.255,1.9,...,-0.1,1.2,0.030,NaN,-0.9,-2.5,-3.3,-0.6,youngni01,2009


In [65]:
df = process_scraped_table(df, 2008)
print(df.shape)
df.head()

DROPPING: Index(['ws-dum', 'bpm-dum'], dtype='object')
(582, 27)


,pos,age,team_id,g,mp,per,ts_pct,fg3a_per_fga_pct,fta_per_fga_pct,orb_pct,...,ows,dws,ws,ws_per_48,obpm,dbpm,bpm,vorp,player_id,year
player,,,,,,,,,,,,,,,,,,,,,
Alex Acker 2009 2008,SG,26,TOT,25,199,10.4,0.448,0.263,0.132,4.5,...,-0.1,0.1,0.0,-0.011,-1.8,-1.7,-3.6,-0.1,ackeral01,2008
Alex Acker 2009 2008,SG,26,DET,7,20,16.0,0.379,0.364,0.182,0.0,...,0.0,0.0,0.0,0.009,-0.9,4.7,3.8,0.0,ackeral01,2008
Alex Acker 2009 2008,SG,26,LAC,18,179,9.8,0.460,0.246,0.123,5.0,...,-0.1,0.0,0.0,-0.013,-1.9,-2.5,-4.4,-0.1,ackeral01,2008
Hassan Adams 2009 2008,SG,24,TOR,12,52,0.4,0.352,0.000,0.462,2.3,...,-0.2,0.0,-0.2,-0.144,-7.5,-1.9,-9.4,-0.1,adamsha01,2008
Arron Afflalo 2009 2008,SG,23,DET,74,1234,8.9,0.548,0.357,0.237,2.8,...,0.8,1.0,1.8,0.069,-2.8,0.2,-2.6,-0.2,afflaar01,2008


In [113]:
df.isna().sum().sort_values(ascending=False)

ts_pct              4
fg3a_per_fga_pct    4
fta_per_fga_pct     4
tov_pct             3
year                0
ast_pct             0
age                 0
team_id             0
g                   0
mp                  0
per                 0
orb_pct             0
drb_pct             0
trb_pct             0
stl_pct             0
player_id           0
blk_pct             0
usg_pct             0
ows                 0
dws                 0
ws                  0
ws_per_48           0
obpm                0
dbpm                0
bpm                 0
vorp                0
pos                 0
dtype: int64

In [96]:
a = pd.DataFrame([[1,2,3], [4,5,6], [7,8,9]], columns = ['a', 'b', 'a'])
print(a)


SyntaxError: invalid syntax (<ipython-input-96-f188aa55ea8f>, line 3)

In [83]:
data_dir = 'TEST/'
seasons = [2008]
stat_types = ['advanced']
df = scrape(seasons, stat_types, data_dir)


In [55]:
df = process_scraped_table(df, 2008)

NO DROP COLS FOUND
year                0
stl_pct             0
age                 0
team_id             0
g                   0
mp                  0
per                 0
ts_pct              0
fg3a_per_fga_pct    0
fta_per_fga_pct     0
orb_pct             0
drb_pct             0
trb_pct             0
ast_pct             0
blk_pct             0
player_id           0
tov_pct             0
usg_pct             0
ws-dum              0
ows                 0
dws                 0
ws                  0
ws_per_48           0
bpm-dum             0
obpm                0
dbpm                0
bpm                 0
vorp                0
pos                 0
dtype: int64
(451, 29)
pos                        PF
age                        31
team_id                   SAC
g                           6
mp                         51
per                       5.5
ts_pct                   .317
fg3a_per_fga_pct         .000
fta_per_fga_pct          .286
orb_pct                  14.2
drb_pct             

In [58]:
a = df.copy()
a = process_scraped_table(a, 2001)
a.head()

NO DROP COLS FOUND
year                0
stl_pct             0
age                 0
team_id             0
g                   0
mp                  0
per                 0
ts_pct              0
fg3a_per_fga_pct    0
fta_per_fga_pct     0
orb_pct             0
drb_pct             0
trb_pct             0
ast_pct             0
blk_pct             0
player_id           0
tov_pct             0
usg_pct             0
ws-dum              0
ows                 0
dws                 0
ws                  0
ws_per_48           0
bpm-dum             0
obpm                0
dbpm                0
bpm                 0
vorp                0
pos                 0
dtype: int64
(451, 29)
pos                        PF
age                        31
team_id                   SAC
g                           6
mp                         51
per                       5.5
ts_pct                   .317
fg3a_per_fga_pct         .000
fta_per_fga_pct          .286
orb_pct                  14.2
drb_pct             

,pos,age,team_id,g,mp,per,ts_pct,fg3a_per_fga_pct,fta_per_fga_pct,orb_pct,...,dws,ws,ws_per_48,bpm-dum,obpm,dbpm,bpm,vorp,player_id,year
player,,,,,,,,,,,,,,,,,,,,,
Shareef Abdur-Rahim 2008 2008 2008 2001,PF,31,SAC,6,51,5.5,.317,.000,.286,14.2,...,0.0,0.0,-0.005,,-4.4,-2.5,-6.9,-0.1,abdursh01,2001
Arron Afflalo 2008 2008 2008 2001,SG,22,DET,75,970,10.2,.494,.199,.361,4.4,...,1.4,1.9,.092,,-2.6,1.0,-1.5,0.1,afflaar01,2001
Maurice Ager 2008 2008 2008 2001,SG,23,TOT,26,165,1.8,.363,.292,.185,2.8,...,0.0,-0.4,-0.114,,-6.8,-4.2,-11.1,-0.4,agerma01,2001
Blake Ahearn 2008 2008 2008 2001,PG,23,MIA,12,177,9.0,.495,.596,.544,0.7,...,0.1,0.0,-0.006,,-3.5,-1.3,-4.8,-0.1,ahearbl01,2001
LaMarcus Aldridge 2008 2008 2008 2001,PF,22,POR,76,2649,18.5,.523,.006,.257,9.8,...,2.8,6.2,.113,,0.6,-0.4,0.2,1.5,aldrila01,2001


In [71]:
drop_cols = a.columns[]

TypeError: isnull() takes 1 positional argument but 2 were given

In [79]:
df = a.copy()
# print(list(df.columns))
# df = df[[col for col in df if not df[col].nunique()==1]]
# print(list(df.columns))

In [81]:
drop_cols = [col for col in df.columns if df[col].nunique() == 1]
drop_cols

['ws-dum', 'bpm-dum', 'year']

In [ ]:
## example driver

options = Options()
options.headless = False

browser = webdriver.Firefox(executable_path="scraping/drivers/geckodriver", options=options)
browser.get('https://duckduckgo.com')

print('Title: %s' % browser.title)
browser.quit()